In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.181, 0.074, 0.067, 0.101],
    [0.078, 0.055, 0.049, 0.093],
    [0.021, 0.069, 0.035, 0.09],
    [0.033, 0.078, 0.076, 0.08],
    [0.028, 0.081, 0.235, 0.071],
    [0.304, 0.052, 0.112, 0.062],
    [0.038, 0.143, 0.032, 0.131],
    [0.014, 0.095, 0.045, 0.134],
    [0.028, 0.144, 0.136, 0.095],
    [0.014, 0.076, 0.148, 0.081],
    [0.009, 0.132, 0.065, 0.063]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10', 'Site11']
criteria = ['C1', 'C2', 'C3', 'C4']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.15288888, 0.30888675, 0.45773133, 0.080493041])

# NORMALIZATION FUNCTION (min-max normalization)
def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

# CODAS METHOD
def codas(matrix, weights, criteria_types, sites, criteria, tau=0.02):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    weighted_matrix = norm_matrix * weights

    # Negative-ideal solution (worst for each criterion)
    neg_ideal = np.min(weighted_matrix, axis=0)

    # Euclidean and Taxicab distances from negative-ideal
    euclidean = np.sqrt(((weighted_matrix - neg_ideal) ** 2).sum(axis=1))
    taxicab = np.abs(weighted_matrix - neg_ideal).sum(axis=1)

    # Assessment score
    # If difference in Euclidean distance between two alternatives < tau, use Taxicab distance as tie-breaker
    scores = np.zeros(len(sites))
    for i in range(len(sites)):
        count = 0
        for j in range(len(sites)):
            if i == j:
                continue
            diff = euclidean[i] - euclidean[j]
            if diff > tau:
                count += 1
            elif abs(diff) <= tau:
                if taxicab[i] > taxicab[j]:
                    count += 1
        scores[i] = count

    # Alternatively, for ranking, you can use the Euclidean distance directly (higher is better)
    df = pd.DataFrame({
        'Euclidean': euclidean,
        'Taxicab': taxicab,
        'CODAS Score': euclidean,  # For simple ranking, use Euclidean distance
        'Rank': rankdata(-euclidean, method='min')
    }, index=sites).sort_values('Rank')

    return df

result_df = codas(matrix, weights, criteria_types, sites, criteria)
print(result_df)


        Euclidean   Taxicab  CODAS Score  Rank
Site5    0.468184  0.575007     0.468184     1
Site9    0.389693  0.590129     0.389693     2
Site7    0.315475  0.397698     0.315475     3
Site11   0.278716  0.344125     0.278716     4
Site10   0.274527  0.365972     0.274527     5
Site6    0.236462  0.333276     0.236462     6
Site8    0.167893  0.256768     0.167893     7
Site1    0.146736  0.285526     0.146736     8
Site4    0.134250  0.219068     0.134250     9
Site3    0.065743  0.101363     0.065743    10
Site2    0.063645  0.118822     0.063645    11
